In [9]:
import os
import time
from PIL import Image
import imageio
import pickle
from tqdm.notebook import tqdm

import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.cm import get_cmap

# import mogrify

In [10]:
base_dir = os.path.join(os.getcwd(), "../")
# base_dir = os.path.join(os.getcwd(), "drive/MyDrive/CSE291/assignments/assignment3/")
data_dir = os.path.join(base_dir, "data/bottles/")
# data_dir = "../data/bottles"
rgb_dir = os.path.join(data_dir, "rgb")
pose_dir = os.path.join(data_dir, "pose")

In [11]:
def _minify(data_dir, factors=[], resolutions=[]):
    needtoload = False
    
    for r in factors:
        imgdir = os.path.join(data_dir, 'rgb_{}'.format(r))  # assignment3/code/../data/bottles/rgb_8
        
        if not os.path.exists(imgdir):
            needtoload=True
    
    for r in resolutions:
        imgdir = os.path.join(data_dir, 'rgb_{}x{}'.format(r[1], r[0]))
        
        if not os.path.exists(imgdir):
            needtoload = True
    
    if needtoload == False:
        return
    
    from shutil import copy
    from subprocess import check_output
    
    imgdir = os.path.join(data_dir, 'rgb')
    
    imgs = [os.path.join(imgdir, f) for f in sorted(os.listdir(imgdir)) if f.endswith('JPG') or 
            f.endswith('jpg') or f.endswith('png') or f.endswith('PNG') or f.endswith('jpeg')]   # length = 200
    
    imgdir_orig = imgdir
    
    working_dir = os.getcwd()
    
    print("This is printed")

    for r in factors + resolutions:
        if isinstance(r , int):
            name = 'rgb_{}'.format(r)
            resizearg = '{}%'.format(100./r)
        else:
            name = 'rgb_{}x{}'.format(r[1], r[0])
            resizearg = '{}x{}'.format(r[1], r[0])
        
        imgdir = os.path.join(data_dir, name)
        
        if os.path.exists(imgdir):
            continue
    
        print("Minifying", r, data_dir)
        
        os.makedirs(imgdir)
        check_output('cp {}/* {}'.format(imgdir_orig, imgdir), shell=True)
        
        extension = imgs[0].split('.')[-1]
        
        args = ' '.join(['mogrify', '-resize', resizearg, '-format', 'png', '*.{}'.format(extension)])
        
        print(args)
        
        os.chdir(imgdir)
        check_output(args, shell=True)
        
        os.chdir(working_dir)
        
        if extension != 'png':
            check_output('rm {}/*.{}'.format(imgdir, extension), shell=true)
            print("Removed duplicates")
        
        print("Done Minifying !!!")

In [4]:
def _load_data(data_dir, factor=None, width=None, height=None, load_imgs=True):
    
    img0 = []
    for f in sorted(os.listdir(os.path.join(data_dir, 'rgb'))):
        if f.endswith('JPG') or f.endswith('jpg') or f.endswith('png'):
            img0.append(os.path.join(data_dir, 'rgb', f))
            break
    
    sh = imageio.imread(img0[0]).shape   # [800, 800, 4]

    sfx = ''
    if factor is not None:
        sfx = '_{}'.format(factor)
        _minify(data_dir, factors=[factor])
        factor = factor
    elif height is not None:
        factor = sh[0] / float(height)
        width = int(sh[1] / factor)
        _minify(data_dir, resolutions=[[height, width]])
        sfx = '_{}x{}'.format(width, height)
    elif width is not None:
        factor = sh[1] / float(width)
        height = int(sh[1] / factor)
        _minify(data_dir, resolutions=[[height, width]])
        sfx = '_{}x{}'.format(width, height)
    else:
        factor = 1
    
    imgdir = os.path.join(data_dir, 'rgb' + sfx)
    if not os.path.exists(imgdir):
        print(imgdir, "does not exist, returning")
        return
    
    imgfiles = []
    for f in sorted(os.listdir(imgdir)):
        if f.endswith('JPG') or f.endswith('jpg') or f.endswith('png'):
            imgfiles.append(os.path.join(imgdir, f))
    
    if not load_imgs:
        return poses, bds
    
    def imread(f):
        if f.endswith('png'):
            return imageio.imread(f, ignoregamma=True)
        else:
            return imageio.imread(f)
        
    imgs = [imread(f)[..., :3] / 255.0 for f in imgfiles]
    imgs = np.stack(imgs, -1)
    
    return imgs

In [5]:
def load_llff_data(datadir, factor=8, recenter=True, bd_factor=0.75, spherify=False, path_zflat=False):
    imgs = _load_data(data_dir, factor=factor)  # factor=8 downsamples images by 8x
    

In [6]:
load_llff_data(data_dir, 8, recenter=True, bd_factor=.75, spherify=False)

This is printed
Minifying 8 /home/sazim/private/CSE291/assignment3/code/../data/bottles/
mogrify -resize 12.5% -format png *.png


/bin/sh: 1: mogrify: not found


CalledProcessError: Command 'mogrify -resize 12.5% -format png *.png' returned non-zero exit status 127.